# GP2

* **Project:** ADRD-SORL1-Biobanks
* **Version:** Python/3.10
* **Last Updated:** 10-Jul-2025

## Notebook Overview
 Gene characterization, allele freqs, association analysis, burden analysis

## Workspace Resources

In [ ]:
# mount resources
! wb resource mount --id=gp2_tier2_eu_release9_18122024

## Imports

In [4]:
## Import the necessary packages 
import os
import numpy as np
import pandas as pd
import math
import sys
import subprocess
import statsmodels.api as sm
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pathlib

## Package Installs

### PLINK

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha5/plink2_linux_x86_64_20240820.zip
unzip -o plink2_linux_x86_64_20240820.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

### ANNOVAR

In [ ]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

In [9]:
%%capture
%%bash

# Install ANNOVAR: Download resources for annotation

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp47a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20240611 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad41_exome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

### RVTests

In [ ]:
%%bash

#Install RVTESTS: Option 1 (~15min)
if test -e /home/jupyter/tools/rvtests; then

echo "rvtests is already installed"
else
echo "rvtests is not installed"

mkdir /home/jupyter/tools/rvtests
cd /home/jupyter/tools/rvtests

wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz 

tar -zxvf rvtests_linux64.tar.gz
fi

In [11]:
# chmod to make sure you have permission to run the program
! chmod u+x /home/jupyter/tools/plink
! chmod u+x /home/jupyter/tools/plink2
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

## Workbook Setup

In [ ]:
## Edit ancestry label and run this cell and all below for remaining ancestries
ancestry = "EUR"
CHROM = "chr11"
GENE = "SORL1"
WORK_DIR =  f"/home/jupyter/{GENE}_results/{ancestry}_R9"
DATA_DIR = f"/home/jupyter/workspace"

! mkdir -p {WORK_DIR}
%cd {WORK_DIR}

## Make Covariate File

In [ ]:
# Let's load the master key
key = pd.read_csv(f"{DATA_DIR}/gp2_tier2_eu_release9_18122024/clinical_data/master_key_release9_final_vwb.csv")
print(key.shape)
key = key[["GP2ID", "baseline_GP2_phenotype", "biological_sex_for_qc", "age_at_sample_collection", "age_of_onset", "age_at_diagnosis","age_at_last_follow_up", "race_for_qc", "nba_label"]]
key.rename(columns = {"GP2ID": "IID",
                      "baseline_GP2_phenotype":"phenotype",
                                     "biological_sex_for_qc":"SEX", 
                                     "age_at_sample_collection":"AGE",
                                     "race_for_qc":"RACE",
                                     "age_at_diagnosis":"AAD",
                                     "age_at_last_follow_up":"AAFU",
                                     "age_of_onset":"AAO"}, inplace = True)

## Subset to keep ancestry of interest 
ancestry_key = key[key["nba_label"]==ancestry].copy()
ancestry_key.reset_index(drop=True)

In [ ]:
# Load information about related individuals 
related_df = pd.read_csv(f"{DATA_DIR}/gp2_tier2_eu_release9_18122024/meta_data/related_samples/{ancestry}_release9_vwb.related")
print(related_df.shape)

# Make a list of just one set of related people
related_list = list(related_df["IID1"].str.rstrip("_s1"))
related_list
# Check value counts of related and remove only one related individual
ancestry_key = ancestry_key[~ancestry_key["IID"].isin(related_list)]

# remove related individuals
print(f"Removing {len(related_list)} individuals.")

# Check size
print(ancestry_key.shape)

In [ ]:
# Convert phenotype to binary (1/2)
## Assign conditions so case=2 and controls=1, and -9 otherwise (matching PLINK convention)
    # PD = 2; control = 1
pheno_mapping = {"PD": 2, "Control": 1}
ancestry_key["PHENO"] = ancestry_key["phenotype"].map(pheno_mapping).astype("Int64")
# Check value counts of pheno
ancestry_key["PHENO"].value_counts(dropna=False)

In [ ]:
# Convert phenotype to binary (1/2)

# Female = 2; Male = 1
sex_mapping = {"Female": 2, "Male": 1}
ancestry_key["SEX"] = ancestry_key["SEX"].map(sex_mapping).astype("Int64")

# Check value counts of SEX
ancestry_key["SEX"].value_counts(dropna=False)

In [ ]:
# only include samples that are in the PLINK files

genotyped_samples = pd.read_csv(f"{DATA_DIR}/gp2_tier2_eu_release9_18122024/imputed_genotypes/{ancestry}/{CHROM}_{ancestry}_release9_vwb.psam", sep = "\t")["#IID"]
genotyped_samples = list(genotyped_samples)

print(f"""Dropping {(~ancestry_key["IID"].isin(genotyped_samples)).sum()} samples without genotyping""")
ancestry_key = ancestry_key[ancestry_key["IID"].isin(genotyped_samples)]
ancestry_key.shape

In [ ]:
## Get the PCs
pcs = pd.read_csv(f"{DATA_DIR}/gp2_tier2_eu_release9_18122024/raw_genotypes/{ancestry}/{ancestry}_release9_vwb.eigenvec", sep = "\t")
selected_columns = ["IID", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9","PC10"]
pcs = pd.DataFrame(data=pcs.iloc[:, 1:12].values, columns=selected_columns)
pcs["IID"] = pcs["IID"].str.replace("_s1", "", case= True)

# Reset the index to remove any potential issues
pcs = pcs.reset_index(drop=True)

# Display the resulting DataFrame
print(pcs)

In [ ]:
## Make covariate file
df = pd.merge(ancestry_key, pcs, on="IID", how= "inner")

## Drop lines with missing pheno
df = df[df["PHENO"].notna()]

print(f"""Dropping {df["PHENO"].isna().sum()} with missing pheno """)
df.shape

In [ ]:
df.columns

In [101]:
df["FID"] = 0

In [ ]:
## Clean up and keep columns we need 
final_df = df[["FID","IID", "SEX", "AGE", "AAO", "PHENO", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9","PC10"]].copy()
final_df.groupby(["PHENO"])["SEX"].value_counts(dropna=False)

In [ ]:
## Make file of sample IDs to keep 
samples_toKeep = final_df[["FID", "IID"]].copy()

samples_toKeep.to_csv(f"{WORK_DIR}/{ancestry}.samplestoKeep", sep = "\t", index=False, header=None)
samples_toKeep.shape

In [ ]:
# Make file of just cases to keep 
cases_toKeep = final_df[final_df["PHENO"]==2][["FID", "IID"]]
cases_toKeep.to_csv(f"{WORK_DIR}/{ancestry}.cases_toKeep", sep = "\t", index = False, header=None)
cases_toKeep.shape

In [105]:
## Save your covariate file
final_df.to_csv(f"{WORK_DIR}/{ancestry}_covariate_file.txt", sep = "\t", index=False, header=True, na_rep="NA")

In [ ]:
final_df

### Get cohort summary stats

In [ ]:
### 
print(f"""
Number of {ancestry} PD cases: {(final_df["PHENO"] == 2).sum()}
Number of {ancestry} Controls: {(final_df["PHENO"] == 1).sum()}

{final_df.groupby(["PHENO"])["SEX"].value_counts(sort=True)} 

PD age range: {final_df[final_df["PHENO"] == 2]["AGE"].mean().round(2)} +/- {round(final_df[final_df["PHENO"] == 2]["AGE"].std(),2)}
Controls age range: {final_df[final_df["PHENO"] == 1]["AGE"].mean().round(2)} +/- {round(final_df[final_df["PHENO"] == 1]["AGE"].std(),2)}

nan_counts:\n{final_df.isna().sum()}
""")


In [ ]:
# extract region of interest from file
! /home/jupyter/tools/plink2 \
--pfile {DATA_DIR}/gp2_tier2_eu_release9_18122024/imputed_genotypes/{ancestry}/{CHROM}_{ancestry}_release9_vwb \
--chr 11 \
--from-bp 121352314 \
--to-bp 121733763 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--make-pgen erase-dosage \
--out {WORK_DIR}/{CHROM}_{ancestry}

## Annotation using ANNOVAR
- *SORL1* from NCBI gene, with gene region flanked by 100kb on both sides
- hg38 (chr11:121452314-121633763) 

### Annotate Variants for All Cases and Controls

In [109]:
# set file prefix for vcf with all cases and controls
PLINK_PREFIX = f"{CHROM}_{GENE}"

In [ ]:
## extract region of interest using plink and Convert the files to VCF format
! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/{CHROM}_{ancestry} \
--chr 11 \
--from-bp 121452314 \
--to-bp 121633763 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--recode vcf id-paste=iid \
--mac 2 \
--out {WORK_DIR}/{PLINK_PREFIX}

In [111]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{PLINK_PREFIX}.vcf
! tabix -f -p vcf {WORK_DIR}/{PLINK_PREFIX}.vcf.gz

In [ ]:
# annotating just one sample id to make annotation run faster
samples = pd.read_csv(f"{WORK_DIR}/{ancestry}.samplestoKeep", sep = "\t", header=None)
sample_id = samples.iloc[0,1]
filename = f"{WORK_DIR}/{PLINK_PREFIX}.vcf.gz"
sample_id

In [ ]:
### export only one sample id
! /home/jupyter/tools/plink2 \
--vcf {filename} \
--indv {sample_id} \
--export vcf bgz \
--out {WORK_DIR}/{ancestry}_{GENE}_{sample_id}

In [114]:
PREFIX = f"{ancestry}_{GENE}_{sample_id}"

In [ ]:
%%bash -s $PREFIX $GENE
## annotate using ANNOVAR

for i in $1
do
    /home/jupyter/tools/annovar/table_annovar.pl ${i}.vcf.gz /home/jupyter/tools/annovar/humandb/ --buildver hg38 --out ${i}.annovar --remove --protocol refGene,dbnsfp47a --operation g,f --otherinfo --polish --nastring . --vcfinput
done

In [ ]:
# check file was made correctly
! ls $PREFIX*multianno*

In [117]:
# rename annovar multianno to remove subject id
! mv -f {PREFIX}.annovar.hg38_multianno.txt {ancestry}_{GENE}.annovar.hg38_multianno.txt
! mv -f {PREFIX}.annovar.hg38_multianno.vcf {ancestry}_{GENE}.annovar.hg38_multianno.vcf

In [ ]:
## Keep only columns of interest to make a manageable file 
! cut -f 1-10,102 {WORK_DIR}/{ancestry}_{GENE}.annovar.hg38_multianno.txt > {WORK_DIR}/{ancestry}_{GENE}.annovar.hg38_multianno.subset.txt
! head {WORK_DIR}/{ancestry}_{GENE}.annovar.hg38_multianno.subset.txt

In [119]:
# Read in subsetted ANNOVAR multianno file
gene = pd.read_csv(f"{WORK_DIR}/{ancestry}_{GENE}.annovar.hg38_multianno.subset.txt", sep = "\t")

In [ ]:
gene["Gene.refGene"].value_counts()

In [121]:
# Keep only annotated ITSN1 variants
gene_subset = gene[gene["Gene.refGene"].isin(["SORL1", "SC5D,SORL1", "SORL1,MIR100HG"])].copy()

In [122]:
gene_subset[["Func.refGene", "ExonicFunc.refGene"]].value_counts().to_csv(f"{WORK_DIR}/{ancestry}_{GENE}_variant_counts.txt", index=True, sep="\t")

In [ ]:
! cat {WORK_DIR}/{ancestry}_{GENE}_variant_counts.txt

In [ ]:
gene_subset

In [125]:
## Filter intronic
intronic = gene_subset[(gene_subset["Func.refGene"] == "intronic")]

In [126]:
## Filter UTR3
utr3 = gene_subset[(gene_subset["Func.refGene"] == "UTR3")]

In [127]:
## Filter UTR5
utr5 = gene_subset[(gene_subset["Func.refGene"] == "UTR5")]

In [128]:
## Filter exonic and synonymous variants
coding_synonymous = gene_subset[(gene_subset["Func.refGene"] == "exonic") & (gene_subset["ExonicFunc.refGene"] == "synonymous SNV")]

In [129]:
# Filter exonic and non-synonymous variants
coding_nonsynonymous = gene_subset[(gene_subset["Func.refGene"]== "exonic") & (gene_subset["ExonicFunc.refGene"]=="nonsynonymous SNV")]

In [130]:
# Filter splicing and stopgain
splicing = gene_subset[(gene_subset["Func.refGene"] == "splicing")]
stopgain = gene_subset[(gene_subset["Func.refGene"] == "exonic") & (gene_subset["ExonicFunc.refGene"] == "stopgain")]

In [131]:
# combine splicing and stopgain variants
all_nonsynonymous = pd.concat([coding_nonsynonymous, splicing, stopgain], axis=0, ignore_index=True)
all_nonsynonymous

# ensure splicing and stopgain labels are in annotations
all_nonsynonymous["ExonicFunc.refGene"] = all_nonsynonymous.apply(lambda row: row["Func.refGene"] if row["ExonicFunc.refGene"] == "." else row["ExonicFunc.refGene"], axis=1)

In [132]:
# Count total variants and subtypes
total_variants = len(gene_subset)
total_intronic = len(intronic)
total_utr3 = len(utr3)
total_utr5 = len(utr5)
total_exonic_syn = len(coding_synonymous)
total_exonic_nonsyn = len(coding_nonsynonymous)
total_splicing = len(splicing)
total_stopgain = len(stopgain)

In [ ]:
# Print summary of counts
print(f"Total Variants in {GENE}: {total_variants:,}")
print(f"    - Intronic: {total_intronic:,}")
print(f"    - UTR3: {total_utr3:,}")
print(f"    - UTR5: {total_utr5:,}")
print(f"    - Exonic Synonymous: {total_exonic_syn:,}")
print(f"    - Exonic Non-synonymous: {total_exonic_nonsyn:,}")
print(f"    - Splicing: {total_splicing:,}")
print(f"    - Stopgain: {total_stopgain:,}")

In [ ]:
all_nonsynonymous["SNP"] = "chr11:" + all_nonsynonymous["Start"].astype(str) + ":" + all_nonsynonymous["Ref"] + ":" + all_nonsynonymous["Alt"]
all_nonsynonymous

In [135]:
# make sure CADD values are numeric
all_nonsynonymous["CADD_phred"] = pd.to_numeric(all_nonsynonymous["CADD_phred"])

In [ ]:
# Save IDS to PLINK format 
variants_toKeep = all_nonsynonymous["SNP"].copy()
variants_toKeep.to_csv(f"{WORK_DIR}/{ancestry}_{GENE}.all_coding_nonsyn.variantstoKeep.txt", sep="\t", index=False, header=False)
variants_toKeep.head()

In [137]:
# save annotation file for adding annotations later
all_nonsynonymous.to_csv(f"{WORK_DIR}/{ancestry}_{GENE}_coding_nonsyn_variant_annotations.txt", sep="\t", index=False)

In [ ]:
## check to make sure file was created and saved
! ls {WORK_DIR}

## Burden Analyses using RVTests


In [ ]:
# get hg38 refFlat file from ucsc
! wget -nc -O /home/jupyter/refFlat.txt.gz https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/refFlat.txt.gz
! gunzip -q -f /home/jupyter/refFlat.txt.gz

In [140]:
# make a pheno file for plink input
! cut -f 1,2,6 {WORK_DIR}/{ancestry}_covariate_file.txt > {WORK_DIR}/{ancestry}_pheno.txt

In [ ]:
# Convert the files from Plink 2.0 to Plink 1.9 format 
! /home/jupyter/tools/plink2 --pfile {WORK_DIR}/{CHROM}_{ancestry} \
--make-bed \
--pheno {ancestry}_pheno.txt \
--no-psam-pheno \
--pheno-name PHENO \
--real-ref-alleles \
--mac 2 \
--max-alleles 2 \
--out {WORK_DIR}/{CHROM}_{ancestry}

In [ ]:
## extract variants
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{CHROM}_{ancestry} \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract {WORK_DIR}/{ancestry}_{GENE}.all_coding_nonsyn.variantstoKeep.txt \
--real-ref-alleles \
--recode vcf-iid \
--out {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn

In [143]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.vcf.gz

In [ ]:
# create cov file that matches rvtest pheno file header
rv_df = pd.read_csv(f"{WORK_DIR}/{ancestry}_covariate_file.txt", sep="\t")
rv_df.columns = rv_df.columns.str.lower()
rv_df["fatid"] = 0
rv_df["matid"] = 0
rv_df = rv_df[["fid", "iid", "fatid", "matid", "sex", "pheno", "age", "pc1", "pc2", "pc3", "pc4", "pc5","pc6","pc7","pc8","pc9","pc10"]]
rv_df.to_csv(f"{WORK_DIR}/rvtests_covariate_file.txt", sep="\t", index=False, header=True)
rv_df["age"].isna().sum()

In [ ]:
## RVtests with covariates 
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--out {WORK_DIR}/{ancestry}_{GENE}.burden.coding_nonsyn \
--kernel skat,skato \
--inVcf {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.vcf.gz \
--pheno {WORK_DIR}/rvtests_covariate_file.txt \
--pheno-name pheno \
--gene {GENE} \
--geneFile /home/jupyter/refFlat.txt \
--covar {WORK_DIR}/rvtests_covariate_file.txt \
--covar-name sex,age,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10

# --out : Name of output 
# --burden cmc --kernel skato: tests to run 
# --inVcf : VCF file 
# --gene: gene name (if only looking at one or a few)
# --geneFile refFlat.txt
# --pheno :  covar file
# --mpheno : # column that has phenotype information
# --pheno-name : column name with phenotype in file
# --covar : covar file
# --freqUpper : optional, MAF cut-off
# --covar-name : covariates, listed by column name, separated by commas (no spaces between commas)
## 1=controls; 2=cases

In [ ]:
## look at results 
! cat {WORK_DIR}/{ancestry}_{GENE}.burden.coding_nonsyn.Skat.assoc

In [ ]:
! cat {WORK_DIR}/{ancestry}_{GENE}.burden.coding_nonsyn.SkatO.assoc

In [ ]:
## check to make sure file was created and saved
! ls {WORK_DIR}

## Case/Control Frequencies

In [149]:
# create new phenotype file (trying to fix issue with plink not reading phenotype column as C/C)
#! cut -f 1,2,6 {WORK_DIR}/{ancestry}_covariate_file.txt > {WORK_DIR}/{ancestry}_pheno_file.txt
#! head {WORK_DIR}/{ancestry}_pheno_file.txt

In [ ]:
## run association test for F_A and F_U
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{CHROM}_{ancestry} \
--extract {WORK_DIR}/{ancestry}_{GENE}.all_coding_nonsyn.variantstoKeep.txt \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_pheno.txt \
--mpheno 1 \
--assoc \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn

In [ ]:
# run logistic regression for pvals and odds ratios
! /home/jupyter/tools/plink2 \
--vcf {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.vcf.gz \
--adjust \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_pheno.txt \
--ci 0.95 \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--covar-variance-standardize \
--glm hide-covar omit-ref firth-fallback cols=+a1freq,+a1freqcc,+a1count,+totallele,+a1countcc,+totallelecc,+gcountcc,+err \
--out {WORK_DIR}/{ancestry}_{GENE}

In [ ]:
# make bfiles with correct phenotypes
! /home/jupyter/tools/plink2 \
--pfile {CHROM}_{ancestry} \
--make-bed \
--max-alleles 2 \
--pheno {WORK_DIR}/{ancestry}_pheno.txt \
--fa /home/jupyter/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna \
--pheno-name PHENO \
--out {WORK_DIR}/{CHROM}_{ancestry}_wPheno

In [ ]:
# generate recode file
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/{CHROM}_{ancestry}_wPheno \
--extract {WORK_DIR}/{ancestry}_{GENE}.all_coding_nonsyn.variantstoKeep.txt \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--recode A \
--out {WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn

In [ ]:
recode = pd.read_csv(f"{WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.raw", sep="\s+")
recode.head()

In [ ]:
### merge output files

ASSOC_FILE = f"{WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.assoc"
RECODE_FILE = f"{WORK_DIR}/{ancestry}_{GENE}.coding_nonsyn.raw"
GLM_HYBRID_FILE = f"{WORK_DIR}/{ancestry}_{GENE}.PHENO.glm.logistic.hybrid"
GLM_ADJUST_FILE = f"{WORK_DIR}/{ancestry}_{GENE}.PHENO.glm.logistic.hybrid.adjusted"

log_hybrid = pd.read_csv(GLM_HYBRID_FILE, sep="\s+", usecols=["ID", "A1", "A1_FREQ", "OBS_CT", "P", "OR", "LOG(OR)_SE", "L95", "U95"])
log_hybrid.rename(columns={"ID":"SNP"}, inplace=True)

assoc_adjusted = pd.read_csv(GLM_ADJUST_FILE,  sep="\s+", usecols=["ID", "BONF"])
assoc_adjusted.rename(columns={"ID":"SNP"}, inplace=True)

assoc = pd.read_csv(ASSOC_FILE, sep="\s+", usecols=["SNP", "F_A", "F_U"])

df = pd.merge(log_hybrid, assoc_adjusted, on="SNP", how="right")

#merge freq with df
freq_assoc = pd.merge(assoc, df, on="SNP", how="left")

#read in recode file
recode = pd.read_csv(RECODE_FILE, sep="\s+")

# Pre-filter the dataset
cases_data = recode[recode["PHENOTYPE"] == 2]
controls_data = recode[recode["PHENOTYPE"] == 1]
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]
results = []
for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    results.append({
        "Variant": variant,
        "Hom Cases": hom_cases,
        "Het Cases": het_cases,
        "Total Cases": total_cases,
        "Hom Controls": hom_controls,
        "Het Controls": het_controls,
        "Total Controls": total_controls,
    })

# Return results
df_results = pd.DataFrame(results)
df_results["SNP"] = df_results["Variant"].apply(lambda x: x.rsplit("_", 1)[0])
df_results = df_results.drop("Variant", axis=1)

# Merge with assoc results 
full_results = pd.merge(freq_assoc, df_results, on="SNP", how="left")

# get BONF threshold value and append to df
full_results["Bonferroni Threshold"] = 0.05/len(full_results["SNP"])

##  get r2 for imputation 
! grep -v "##" {WORK_DIR}/{CHROM}_{ancestry}.pvar > {WORK_DIR}/{CHROM}_{ancestry}_imputation_info.txt

variant_file = pd.read_csv(f"{WORK_DIR}/{CHROM}_{ancestry}_imputation_info.txt", sep ="\t")
R2 = variant_file["INFO"].str.extract("(R2=\d+(\.\d+)?)", expand=False)[0].str.strip()

variant_file["R2"] = R2.str.strip("R2=")
variant_file["SNP"] = variant_file["ID"]
# make df with variant R2
variant_df = variant_file[["SNP", "R2"]].copy()

# append R2 to the final results dataframe
full_results_with_r = pd.merge(full_results, variant_df, on="SNP", how="left")
full_results_with_r

#subset to only columns to keep
clean_full_results = full_results_with_r[["SNP", "A1", "A1_FREQ", "P", "OR", "LOG(OR)_SE","L95", "U95", "BONF", 
                                   "F_A", "F_U",
                                   "Hom Cases", "Het Cases", "Total Cases", 
                                   "Hom Controls","Het Controls", "Total Controls",
                                  "Bonferroni Threshold", "R2"]].copy()
clean_full_results

In [156]:
# add ancestry 
clean_full_results.insert(0, "Ancestry", ancestry)

In [ ]:
print(clean_full_results.shape)
print(f"No. of SNPs: {clean_full_results.shape[0]}")
clean_full_results

### Keep variants only in cases with CADD > 20

In [ ]:
# get variants that are only in cases
cases_only = clean_full_results[(clean_full_results["Hom Controls"] + clean_full_results["Het Controls"]) == 0].copy()
cases_only

annotations = pd.read_csv(f"{WORK_DIR}/{ancestry}_{GENE}_coding_nonsyn_variant_annotations.txt", sep="\t", usecols=["SNP", "CADD_phred", "ExonicFunc.refGene"])

cases_only = cases_only.merge(annotations, on="SNP")
cases_only


In [ ]:
# keep case only variants with CADD score > 20
cases_only_cadd20 = cases_only[cases_only["CADD_phred"] > 20]
only_in_cases_df = cases_only_cadd20[["Ancestry", "SNP", "ExonicFunc.refGene", "Hom Cases", "Het Cases", "Total Cases", "CADD_phred", "F_A"]].copy()
only_in_cases_df.rename(columns={"F_A":"Case MAF", "ExonicFunc.refGene":"Consequence"}, inplace=True)

# write to file
only_in_cases_df.to_csv(f"{WORK_DIR}/{ancestry}_{GENE}_possibly_pathogenic_variants.txt", index=False, header=True, sep="\t")
only_in_cases_df

## Finalize Results Files

In [ ]:
# append variant consequences
clean_full_results_annotated = clean_full_results.merge(annotations, on="SNP", how="left")
clean_full_results_annotated.rename(columns={"ExonicFunc.refGene":"Consequence"}, inplace=True)
col = clean_full_results_annotated.pop("Consequence")
clean_full_results_annotated.insert(1, "Consequence", col)
clean_full_results_annotated

In [ ]:
# Look at significant SNPs, if any
sig_freq = clean_full_results_annotated[clean_full_results_annotated["P"]<0.05]
sig_snps = sig_freq["SNP"].tolist()
sig_df_results = clean_full_results_annotated[clean_full_results_annotated["SNP"].isin(sig_snps)]
sig_df_results

In [162]:
# save files to working directory
clean_full_results_annotated.to_csv(f"{WORK_DIR}/{ancestry}_{GENE}_GP2_R9.fullVariantInformation.txt", sep="\t", index=False)
sig_df_results.to_csv(f"{WORK_DIR}/{ancestry}_{GENE}_GP2_R9.SignificantVariantInformation.txt" , sep="\t", index=False)